In [1]:
import tensorflow as tf
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# hyperparameters
batch_size = 64 # how many independent sequences will we process in parallel?
block_size = 128 # what is the maximum context length for predictions?
max_iters = 2000
eval_interval = 100
learning_rate = 3e-3
eval_iters = 200
n_embd = 384
n_head = 4
n_layer = 4
dropout = 0.2
# ------------

In [2]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

In [3]:
# Train and test splits
data = tf.convert_to_tensor(encode(text), dtype=tf.int64)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [4]:
g1 = tf.random.Generator.from_seed(1331)

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = g1.uniform(minval=0,maxval=len(data)-block_size,shape=[batch_size,],dtype=tf.dtypes.int64)
    
    x = tf.map_fn(lambda i:tf.gather(data, tf.range(i,i+block_size)), ix,dtype=tf.dtypes.int64)
    y = tf.map_fn(lambda i:tf.gather(data, tf.range(i+1,i+block_size+1)), ix,dtype=tf.dtypes.int64)

    return tf.stack(x), tf.stack(y)

In [5]:
def estimate_loss():
    out = {}
    for split in ['train', 'val']:
        losses =[0]*eval_iters
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.numpy().tolist()
        out[split] = tf.math.reduce_mean(tf.constant(losses))
    return out

In [6]:
class Head(layers.Layer):
    """ one head of self-attention """

    def __init__(self, head_size):
        super(Head, self).__init__()
        self.key = layers.Dense(head_size, use_bias=False,input_shape=(None,n_embd))
        self.query = layers.Dense(head_size, use_bias=False,input_shape=(None,n_embd))
        self.value = layers.Dense(head_size, use_bias=False,input_shape=(None,n_embd))
        self.tril = tf.linalg.LinearOperatorLowerTriangular(tf.ones((block_size, block_size)))
        self.dropout = layers.Dropout(dropout)

    def call(self, x):
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        B,T,C = x.shape
        k = self.key(x)   # (B,T,hs)
        q = self.query(x) # (B,T,hs)
        # compute attention scores ("affinities")
        t = tf.transpose(k, perm=[0, 2, 1])
        wei = q @ t * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        wei = tf.where(tf.equal(self.tril.to_dense()[None, :T, :T], 0), float('-inf'), wei) # (B, T, T)
        wei = tf.nn.softmax(wei,axis=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,hs)
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out


In [7]:
class MultiHeadAttention(layers.Layer):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super(MultiHeadAttention, self).__init__()
        self.heads = [Head(head_size) for _ in range(num_heads)]
        self.proj = layers.Dense(n_embd,input_shape=(None,head_size * num_heads))
        self.dropout = layers.Dropout(dropout)

    def call(self, x):
        head_outputs = [head(x) for head in self.heads]
        out = tf.concat(head_outputs, axis=-1)
        out = self.dropout(out)
        out = self.proj(out)
        return out

In [8]:
from tensorflow.keras import Sequential
class FeedFoward(layers.Layer):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super(FeedFoward,self).__init__()
        self.net = Sequential([layers.Dense(4 * n_embd),
                               layers.ReLU(),
                               layers.Dense(n_embd),
                               layers.Dropout(dropout)
                              ])

    def call(self, x):
        return self.net(x)

In [9]:
class Block(layers.Layer):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super(Block,self).__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = layers.LayerNormalization()
        self.ln2 = layers.LayerNormalization()

    def call(self, x):
        s = self.sa(self.ln1(x))
        x = x + s
        f = self.ffwd(self.ln2(x))
        x = x + f
        return(x)

In [27]:
class GPTLanguageModel(layers.Layer):

    def __init__(self):
        super(GPTLanguageModel, self).__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = layers.Embedding(vocab_size, n_embd,name="em1")
        self.position_embedding_table = layers.Embedding(block_size, n_embd,name="em2")
        self.blocks = keras.Sequential([Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = layers.LayerNormalization() # final layer norm
        self.lm_head = layers.Dense(vocab_size,input_shape=(None,4 * n_embd),name="dense1")

        # better init, not covered in the original GPT video, but important, will cover in followup video
        self._init_weights()
        

    
    def _init_weights(self):
        
        for module in self.submodules:
            if isinstance(module, layers.Dense):
                module.kernel_initializer = keras.initializers.RandomNormal(mean=0.0, stddev=0.02)
                if module.use_bias:
                    module.bias_initializer = keras.initializers.Zeros()
            elif isinstance(module, layers.Embedding):
                module.embeddings_initializer = keras.initializers.RandomNormal(mean=0.0, stddev=0.02)
    


    def call(self, idx, targets=None):
        
        B, T = idx.shape
        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(tf.range(T, dtype=tf.int32)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = tf.reshape(logits, (B*T, C))
            targets = tf.reshape(targets, B*T)
            loss = tf.keras.losses.sparse_categorical_crossentropy(targets, logits, from_logits=True)
            loss = tf.reduce_mean(loss)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, _ = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :]  # becomes (B, C)
            # apply softmax to get probabilities
            probs = tf.nn.softmax(logits, axis=-1)  # (B, C)
            # sample from the distribution
            idx_next = tf.random.categorical(probs, num_samples=1, dtype=tf.int32)  # (B, 1)
            # append sampled index to the running sequence
            idx = tf.concat([idx, idx_next], axis=1)  # (B, T+1)
        return idx
    
    def saveModel():
        x = model.weights
        folder = "model/"
        for i in x:
            fileName = str(i.name.replace('/','__').replace(':','##')) +".txt"
            f = open(str(folder+fileName),"w")
            numpy.savetxt(f,i)
            f.close()

In [28]:
model = GPTLanguageModel()

In [ ]:


for i in x:
    modelName = str(i.split('.txt')[0].replace('__','/').replace('##',':'))
    f = open(i,"r")
    modelData = numpy.loadtext(f,i)
    model.set_weights()
    f.close()

In [33]:
model.submodules[4].name

'dense_121'

In [ ]:
import os
x = os.listDir("/model/")

def loadModel(m):
    for layer in m.submodules:
        if isinstance(layer, tf.keras.models.Model):
            loadModel(layer)
        else:
            f = open()
            weights = numpy.loadtxt(f)
            layer.set_weights(weights)
        
        

In [38]:
from tensorflow.python.util import tf_export
from tensorflow.python.util.tf_export import keras_export

In [49]:
for i in model.submodules:
    x = tf_export.get_canonical_name_for_symbol(
            i.__class__, api_name="keras"
        )
    parts = x.split(".")
    module = ".".join(parts[:-1])
    class_name = parts[-1]
    registered_name = None
    config = {
        "module": module,
        "class_name": class_name,
        "registered_name": registered_name,
    }
    if hasattr(i, "get_build_config"):
        build_config = i.get_build_config()
        if build_config is not None:
            config["build_config"] = serialize_dict(build_config)
    if hasattr(i, "get_compile_config"):
        compile_config = i.get_compile_config()
        if compile_config is not None:
            config["compile_config"] = serialize_dict(compile_config)
    
    print(config)

{'class_name': 'Embedding', 'module': 'keras.layers', 'registered_name': None}
{'class_name': 'Embedding', 'module': 'keras.layers', 'registered_name': None}
{'class_name': 'Sequential', 'module': 'keras', 'registered_name': None}
{'class_name': 'LayerNormalization',
 'module': 'keras.layers',
 'registered_name': None}
{'class_name': 'Dense', 'module': 'keras.layers', 'registered_name': None}


AttributeError: 'NoneType' object has no attribute 'split'

In [43]:
model.__class__

__main__.GPTLanguageModel